# G1 Humanoid Spinkick Training on Google Colab

This notebook trains a Unitree G1 humanoid robot to perform spin kicks using motion imitation learning.

**Runtime:** Use GPU runtime (Runtime -> Change runtime type -> GPU -> T4)

In [ ]:
# Check GPU availability
!nvidia-smi
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

## 1. Setup Virtual Display and Install Dependencies

In [ ]:
# Install virtual display and rendering dependencies
!apt-get update -qq
!apt-get install -y xvfb x11-utils
!apt-get install -y libgl1-mesa-glx libgl1-mesa-dri
!apt-get install -y libglu1-mesa libglib2.0-0 libsm6 libxrender1 libxext6
!apt-get install -y ffmpeg

# Install Python dependencies
!pip install mujoco
!pip install gymnasium
!pip install wandb
!pip install moviepy imageio imageio-ffmpeg
!pip install stable-baselines3
!pip install tensorboard
!pip install pyvirtualdisplay

# Set up virtual display
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()

import os
os.environ['DISPLAY'] = ':' + str(display.display)

# Test MuJoCo installation
import mujoco
print(f"MuJoCo version: {mujoco.__version__}")
print(f"Virtual display started on {os.environ['DISPLAY']}")

## 2. Setup WandB (Optional)

In [ ]:
# Login to WandB (optional for tracking)
import wandb
# wandb.login(key="YOUR_API_KEY")  # Uncomment and add your key

# Or run without WandB
wandb_available = False
try:
    wandb.init(project="g1-spinkick-colab", mode="disabled")  # Disabled by default
    wandb_available = True
except:
    print("Running without WandB tracking")

## 3. Create G1 Robot Model

In [ ]:
# Define a simplified G1 humanoid model
g1_xml = """
<mujoco model="g1_simplified">
  <option timestep="0.002" gravity="0 0 -9.81"/>
  
  <default>
    <joint armature="0.1" damping="1" limited="true"/>
    <geom contype="1" conaffinity="1" condim="3" friction="0.8 0.1 0.1"/>
  </default>
  
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .1 .1" rgb2=".2 .2 .2" width="300" height="300"/>
    <material name="grid" texture="grid" texrepeat="1 1" texuniform="true" reflectance=".2"/>
  </asset>
  
  <worldbody>
    <!-- Ground -->
    <geom name="floor" type="plane" size="10 10 0.1" material="grid"/>
    
    <!-- G1 Robot (simplified) -->
    <body name="pelvis" pos="0 0 0.9">
      <joint name="root_x" type="slide" axis="1 0 0" limited="false"/>
      <joint name="root_y" type="slide" axis="0 1 0" limited="false"/>
      <joint name="root_z" type="slide" axis="0 0 1" limited="false"/>
      <joint name="root_roll" type="hinge" axis="1 0 0" limited="false"/>
      <joint name="root_pitch" type="hinge" axis="0 1 0" limited="false"/>
      <joint name="root_yaw" type="hinge" axis="0 0 1" limited="false"/>
      
      <geom name="pelvis_geom" type="box" size="0.15 0.1 0.1" rgba="0.8 0.3 0.3 1"/>
      <inertial pos="0 0 0" mass="10" diaginertia="0.1 0.1 0.1"/>
      
      <!-- Torso -->
      <body name="torso" pos="0 0 0.2">
        <joint name="waist_pitch" type="hinge" axis="0 1 0" range="-0.5 0.5"/>
        <joint name="waist_yaw" type="hinge" axis="0 0 1" range="-1 1"/>
        <geom name="torso_geom" type="box" size="0.12 0.08 0.15" rgba="0.3 0.3 0.8 1"/>
        <inertial pos="0 0 0" mass="8" diaginertia="0.08 0.08 0.08"/>
        
        <!-- Arms (simplified) -->
        <body name="left_arm" pos="0.12 0 0.1">
          <joint name="left_shoulder" type="ball" limited="false"/>
          <geom name="left_arm_geom" type="capsule" size="0.03" fromto="0 0 0 0.3 0 -0.2"/>
          <inertial pos="0.15 0 -0.1" mass="2" diaginertia="0.02 0.02 0.02"/>
        </body>
        
        <body name="right_arm" pos="-0.12 0 0.1">
          <joint name="right_shoulder" type="ball" limited="false"/>
          <geom name="right_arm_geom" type="capsule" size="0.03" fromto="0 0 0 -0.3 0 -0.2"/>
          <inertial pos="-0.15 0 -0.1" mass="2" diaginertia="0.02 0.02 0.02"/>
        </body>
      </body>
      
      <!-- Left Leg -->
      <body name="left_hip" pos="0.08 0 -0.05">
        <joint name="left_hip_roll" type="hinge" axis="1 0 0" range="-0.5 0.5"/>
        <joint name="left_hip_pitch" type="hinge" axis="0 1 0" range="-2 1"/>
        <joint name="left_hip_yaw" type="hinge" axis="0 0 1" range="-0.8 0.8"/>
        <geom name="left_thigh_geom" type="capsule" size="0.05" fromto="0 0 0 0 0 -0.35"/>
        <inertial pos="0 0 -0.175" mass="3" diaginertia="0.03 0.03 0.03"/>
        
        <body name="left_knee" pos="0 0 -0.35">
          <joint name="left_knee_pitch" type="hinge" axis="0 1 0" range="-0.1 2.5"/>
          <geom name="left_shin_geom" type="capsule" size="0.04" fromto="0 0 0 0 0 -0.35"/>
          <inertial pos="0 0 -0.175" mass="2" diaginertia="0.02 0.02 0.02"/>
          
          <body name="left_foot" pos="0 0 -0.35">
            <joint name="left_ankle_pitch" type="hinge" axis="0 1 0" range="-0.8 0.8"/>
            <joint name="left_ankle_roll" type="hinge" axis="1 0 0" range="-0.4 0.4"/>
            <geom name="left_foot_geom" type="box" size="0.06 0.03 0.01" rgba="0.2 0.2 0.2 1"/>
            <inertial pos="0 0 0" mass="0.5" diaginertia="0.005 0.005 0.005"/>
          </body>
        </body>
      </body>
      
      <!-- Right Leg (mirror of left) -->
      <body name="right_hip" pos="-0.08 0 -0.05">
        <joint name="right_hip_roll" type="hinge" axis="1 0 0" range="-0.5 0.5"/>
        <joint name="right_hip_pitch" type="hinge" axis="0 1 0" range="-2 1"/>
        <joint name="right_hip_yaw" type="hinge" axis="0 0 1" range="-0.8 0.8"/>
        <geom name="right_thigh_geom" type="capsule" size="0.05" fromto="0 0 0 0 0 -0.35"/>
        <inertial pos="0 0 -0.175" mass="3" diaginertia="0.03 0.03 0.03"/>
        
        <body name="right_knee" pos="0 0 -0.35">
          <joint name="right_knee_pitch" type="hinge" axis="0 1 0" range="-0.1 2.5"/>
          <geom name="right_shin_geom" type="capsule" size="0.04" fromto="0 0 0 0 0 -0.35"/>
          <inertial pos="0 0 -0.175" mass="2" diaginertia="0.02 0.02 0.02"/>
          
          <body name="right_foot" pos="0 0 -0.35">
            <joint name="right_ankle_pitch" type="hinge" axis="0 1 0" range="-0.8 0.8"/>
            <joint name="right_ankle_roll" type="hinge" axis="1 0 0" range="-0.4 0.4"/>
            <geom name="right_foot_geom" type="box" size="0.06 0.03 0.01" rgba="0.2 0.2 0.2 1"/>
            <inertial pos="0 0 0" mass="0.5" diaginertia="0.005 0.005 0.005"/>
          </body>
        </body>
      </body>
    </body>
  </worldbody>
  
  <actuator>
    <!-- Torso actuators -->
    <motor joint="waist_pitch" gear="100"/>
    <motor joint="waist_yaw" gear="100"/>
    
    <!-- Leg actuators -->
    <motor joint="left_hip_roll" gear="150"/>
    <motor joint="left_hip_pitch" gear="150"/>
    <motor joint="left_hip_yaw" gear="150"/>
    <motor joint="left_knee_pitch" gear="150"/>
    <motor joint="left_ankle_pitch" gear="50"/>
    <motor joint="left_ankle_roll" gear="50"/>
    
    <motor joint="right_hip_roll" gear="150"/>
    <motor joint="right_hip_pitch" gear="150"/>
    <motor joint="right_hip_yaw" gear="150"/>
    <motor joint="right_knee_pitch" gear="150"/>
    <motor joint="right_ankle_pitch" gear="50"/>
    <motor joint="right_ankle_roll" gear="50"/>
  </actuator>
</mujoco>
"""

# Save the model
with open('g1_model.xml', 'w') as f:
    f.write(g1_xml)

# Load and test the model
model = mujoco.MjModel.from_xml_string(g1_xml)
data = mujoco.MjData(model)

print(f"Model loaded successfully!")
print(f"Number of joints: {model.njnt}")
print(f"Number of actuators: {model.nu}")
print(f"Degrees of freedom: {model.nv}")

## 4. Create Spinkick Reference Motion

In [ ]:
import numpy as np

def create_spinkick_reference(duration=2.0, fps=50):
    """Create a reference spinkick motion."""
    num_frames = int(duration * fps)
    times = np.linspace(0, duration, num_frames)
    
    reference_motion = []
    
    for t in times:
        phase = t / duration
        
        # Root position (slight jump)
        root_z = 0.9 + 0.2 * np.sin(phase * 2 * np.pi)
        
        # Root rotation (spin)
        root_yaw = phase * 2 * np.pi  # One full rotation
        
        # Leg positions for kick
        if phase < 0.25:  # Preparation
            left_hip_pitch = -0.5
            left_knee_pitch = 1.0
            right_hip_pitch = -0.3
            right_knee_pitch = 0.6
        elif phase < 0.5:  # Kick out
            left_hip_pitch = 0.8  # Kick forward
            left_knee_pitch = 0.1  # Straighten
            right_hip_pitch = -0.5
            right_knee_pitch = 1.5
        elif phase < 0.75:  # Second kick
            left_hip_pitch = -0.5
            left_knee_pitch = 1.5
            right_hip_pitch = 0.8  # Right kick
            right_knee_pitch = 0.1
        else:  # Recovery
            left_hip_pitch = -0.3
            left_knee_pitch = 0.6
            right_hip_pitch = -0.3
            right_knee_pitch = 0.6
        
        # Arm motion (balance)
        arm_angle = np.sin(phase * 4 * np.pi) * 0.5
        
        frame = {
            'time': t,
            'root_pos': [0, 0, root_z],
            'root_rot': [0, 0, root_yaw],
            'joints': {
                'waist_pitch': 0,
                'waist_yaw': 0,
                'left_hip_pitch': left_hip_pitch,
                'left_knee_pitch': left_knee_pitch,
                'right_hip_pitch': right_hip_pitch,
                'right_knee_pitch': right_knee_pitch,
                'left_hip_roll': 0,
                'left_hip_yaw': arm_angle * 0.5,
                'right_hip_roll': 0,
                'right_hip_yaw': -arm_angle * 0.5,
                'left_ankle_pitch': 0,
                'left_ankle_roll': 0,
                'right_ankle_pitch': 0,
                'right_ankle_roll': 0
            }
        }
        reference_motion.append(frame)
    
    return reference_motion

# Create reference motion
reference_motion = create_spinkick_reference()
print(f"Created reference motion with {len(reference_motion)} frames")

## 5. Create G1 Spinkick Environment

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import imageio

class G1SpinkickEnv(gym.Env):
    def __init__(self, reference_motion, render_mode=None):
        super().__init__()
        
        self.model = mujoco.MjModel.from_xml_string(g1_xml)
        self.data = mujoco.MjData(self.model)
        self.reference_motion = reference_motion
        self.render_mode = render_mode
        
        # Action space: actuator commands
        self.action_space = spaces.Box(
            low=-1.0, high=1.0, shape=(self.model.nu,), dtype=np.float32
        )
        
        # Observation space: joint positions, velocities, reference pose
        obs_dim = self.model.nq + self.model.nv + self.model.nq  # pos + vel + ref
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(obs_dim,), dtype=np.float32
        )
        
        self.dt = self.model.opt.timestep
        self.max_episode_steps = len(reference_motion)
        
        # Create renderer for offscreen rendering
        self.renderer = mujoco.Renderer(self.model, height=480, width=640)
        
        # For video recording
        self.frames = []
        self.recording = False
        
    def start_recording(self):
        self.recording = True
        self.frames = []
        
    def stop_recording(self):
        self.recording = False
        return self.frames
        
    def reset(self, seed=None):
        super().reset(seed=seed)
        
        mujoco.mj_resetData(self.model, self.data)
        self.step_count = 0
        
        # Set initial pose from reference
        ref_frame = self.reference_motion[0]
        self.data.qpos[0:3] = ref_frame['root_pos']
        
        # Clear frames if not recording
        if not self.recording:
            self.frames = []
        
        return self._get_obs(), {}
    
    def step(self, action):
        # Apply action
        self.data.ctrl[:] = action * 0.3  # Scale down actions
        
        # Step simulation
        mujoco.mj_step(self.model, self.data)
        self.step_count += 1
        
        # Record frame if requested
        if self.recording:
            self.renderer.update_scene(self.data, camera="track")
            frame = self.renderer.render()
            self.frames.append(frame)
        
        # Get observation and reward
        obs = self._get_obs()
        reward = self._compute_reward()
        
        # Check termination
        terminated = self._is_terminated()
        truncated = self.step_count >= self.max_episode_steps
        
        return obs, reward, terminated, truncated, {}
    
    def _get_obs(self):
        # Current state
        qpos = self.data.qpos.copy()
        qvel = self.data.qvel.copy()
        
        # Reference pose
        ref_idx = min(self.step_count, len(self.reference_motion) - 1)
        ref_frame = self.reference_motion[ref_idx]
        ref_qpos = np.zeros_like(qpos)
        ref_qpos[0:3] = ref_frame['root_pos']
        ref_qpos[5] = ref_frame['root_rot'][2]  # yaw
        
        # Concatenate
        return np.concatenate([qpos, qvel, ref_qpos])
    
    def _compute_reward(self):
        ref_idx = min(self.step_count, len(self.reference_motion) - 1)
        ref_frame = self.reference_motion[ref_idx]
        
        # Position tracking reward
        root_pos = self.data.qpos[0:3]
        ref_pos = ref_frame['root_pos']
        pos_error = np.linalg.norm(root_pos - ref_pos)
        pos_reward = np.exp(-pos_error * 5)
        
        # Rotation tracking reward
        root_yaw = self.data.qpos[5]
        ref_yaw = ref_frame['root_rot'][2]
        rot_error = abs(root_yaw - ref_yaw)
        rot_reward = np.exp(-rot_error * 2)
        
        # Stay upright reward
        upright_reward = self.data.qpos[2]  # Height
        
        # Energy penalty
        energy_penalty = 0.001 * np.sum(self.data.ctrl**2)
        
        return pos_reward + rot_reward + upright_reward - energy_penalty
    
    def _is_terminated(self):
        # Fall detection
        if self.data.qpos[2] < 0.3:  # Too low
            return True
        
        # Extreme joint angles
        if np.any(np.abs(self.data.qpos[6:]) > 3.0):
            return True
        
        return False
    
    def render(self):
        self.renderer.update_scene(self.data)
        return self.renderer.render()

# Test the environment
env = G1SpinkickEnv(reference_motion)
obs, _ = env.reset()
print(f"Environment created!")
print(f"Observation shape: {obs.shape}")
print(f"Action shape: {env.action_space.shape}")

## 6. Train with PPO (Using Stable Baselines3)

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback

class TensorboardCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(TensorboardCallback, self).__init__(verbose)
        self.episode_rewards = []
        self.episode_lengths = []
        
    def _on_step(self) -> bool:
        if self.locals.get('dones')[0]:
            episode_reward = self.locals['infos'][0].get('episode', {}).get('r', 0)
            episode_length = self.locals['infos'][0].get('episode', {}).get('l', 0)
            
            self.episode_rewards.append(episode_reward)
            self.episode_lengths.append(episode_length)
            
            if len(self.episode_rewards) % 10 == 0:
                avg_reward = np.mean(self.episode_rewards[-10:])
                avg_length = np.mean(self.episode_lengths[-10:])
                print(f"Episodes: {len(self.episode_rewards)}, "
                      f"Avg Reward: {avg_reward:.2f}, "
                      f"Avg Length: {avg_length:.0f}")
        return True

# Create vectorized environment
def make_env():
    return G1SpinkickEnv(reference_motion)

vec_env = DummyVecEnv([make_env])

# Create PPO model
model = PPO(
    "MlpPolicy",
    vec_env,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.01,
    verbose=1,
    tensorboard_log="./tensorboard/",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

print(f"Training on: {model.device}")
print("Starting training...")

# Train
callback = TensorboardCallback()
model.learn(
    total_timesteps=100000,  # Adjust based on Colab runtime limits
    callback=callback,
    progress_bar=True
)

## 7. Evaluate and Create Video

In [ ]:
# Evaluate the trained model and create video
env = G1SpinkickEnv(reference_motion)
obs, _ = env.reset()

# Start recording
env.start_recording()

episode_reward = 0
episode_length = 0

print("Recording evaluation episode...")

for step in range(len(reference_motion)):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, _ = env.step(action)
    episode_reward += reward
    episode_length += 1
    
    if terminated or truncated:
        print(f"Episode ended at step {step}")
        break

# Get recorded frames
frames = env.stop_recording()

print(f"Episode length: {episode_length}")
print(f"Total reward: {episode_reward:.2f}")
print(f"Average reward per step: {episode_reward/episode_length:.3f}")
print(f"Recorded {len(frames)} frames")

# Create video
if frames:
    print("Creating video...")
    video_path = "g1_spinkick_evaluation.mp4"
    with imageio.get_writer(video_path, fps=30) as writer:
        for frame in frames:
            writer.append_data(frame)
    
    # Display video in Colab
    from IPython.display import Video
    display(Video(video_path, width=640, height=480))
    print("Video created successfully!")
else:
    print("No frames recorded")

### 7.1 Create Reference Motion Video

In [ ]:
# Create video of the reference motion
print("Creating reference motion video...")

# Reset environment
model_ref = mujoco.MjModel.from_xml_string(g1_xml)
data_ref = mujoco.MjData(model_ref)
renderer_ref = mujoco.Renderer(model_ref, height=480, width=640)

ref_frames = []

for i, ref_frame in enumerate(reference_motion):
    # Set robot pose from reference
    data_ref.qpos[0:3] = ref_frame['root_pos']
    data_ref.qpos[5] = ref_frame['root_rot'][2]  # yaw
    
    # Set joint angles
    for joint_name, angle in ref_frame['joints'].items():
        joint_id = mujoco.mj_name2id(model_ref, mujoco.mjtObj.mjOBJ_JOINT, joint_name)
        if joint_id >= 0:
            joint_qpos_adr = model_ref.jnt_qposadr[joint_id]
            data_ref.qpos[joint_qpos_adr] = angle
    
    # Forward dynamics to update positions
    mujoco.mj_forward(model_ref, data_ref)
    
    # Render frame
    renderer_ref.update_scene(data_ref, camera="track")
    frame = renderer_ref.render()
    ref_frames.append(frame)

# Create reference video
ref_video_path = "g1_spinkick_reference.mp4"
with imageio.get_writer(ref_video_path, fps=30) as writer:
    for frame in ref_frames:
        writer.append_data(frame)

print("Reference motion video:")
display(Video(ref_video_path, width=640, height=480))

## 8. Visualize Training Results

In [ ]:
# Save the trained model
model.save("g1_spinkick_ppo")
print("Model saved!")

# Download the model and videos
from google.colab import files
print("\nDownloading files...")
files.download('g1_spinkick_ppo.zip')
if os.path.exists('g1_spinkick_evaluation.mp4'):
    files.download('g1_spinkick_evaluation.mp4')
if os.path.exists('g1_spinkick_reference.mp4'):
    files.download('g1_spinkick_reference.mp4')

## 9. Save Model

In [ ]:
# Save the trained model
model.save("g1_spinkick_ppo")
print("Model saved!")

# Download the model
from google.colab import files
files.download('g1_spinkick_ppo.zip')